In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = "../data/train.csv"
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
print(np.shape(y))
print(np.shape(tX))
print(np.shape(ids))
print(tX[18,:])

(250000,)
(250000, 30)
(250000,)
[-9.99000e+02  8.51860e+01  6.88270e+01  5.04200e+00 -9.99000e+02
 -9.99000e+02 -9.99000e+02  2.11600e+00  5.04200e+00  7.14430e+01
  1.55800e+00 -1.35100e+00 -9.99000e+02  2.79310e+01  1.17500e+00
  2.35600e+00  4.35120e+01  2.33200e+00  5.84000e-01  4.46980e+01
 -2.03300e+00  1.51816e+02  0.00000e+00 -9.99000e+02 -9.99000e+02
 -9.99000e+02 -9.99000e+02 -9.99000e+02 -9.99000e+02  0.00000e+00]


## Do your thing crazy machine learning thing here :) ...

In [3]:
def  replace_data_point_by_mean(x):# replace missing datapoint of matrix x by the mean of each column
    
    col = x.shape[1]
    row = x.shape[0]
    
    for i in range(col):
        if -999 in x[:,i]:
            ind_nan = []
            compressed =[]
            for j in range (row):
                if x[j,i] == -999:
                    ind_nan.append(j)
                else:
                    compressed.append(x[j,i])
            mean = np.mean(compressed)
            
            x[:,i] = np.where(x[:,i] == -999, mean, x[:,i])
            
    return x
            

In [10]:
from implementations import *
from plots import *

''' bias variance + cross validation + ridge regression ->  get optimal degree and lambda for each parameter
'''

def bias_variance_demo_with_cross_validation_and_ridge(y,x): # param x,y
    """The entry."""
    # define parameters
    seed = 1 #range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambdas = np.logspace(-4, 0, 30)
    k_fold = 4
    
    
    # define list to store the variable
    rmse_tr = np.empty(len(degrees))
    rmse_te = np.empty(len(degrees))
    
    #for index_seed, seed in enumerate(seeds):
    #    np.random.seed(seed)
        #x = np.linspace(0.1, 2 * np.pi, num_data) # to delete
        #y = np.sin(x) + 0.3 * np.random.randn(num_data).T # to delete
        # ***************************************************
        # INSERT YOUR CODE HERE
        # split data with a specific seed:
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio_train, seed)
    k_indices = build_k_indices(y_tr, k_fold, seed)
        # ***************************************************
        # ***************************************************
        # INSERT YOUR CODE HERE
        # bias_variance_decomposition: TODO
    for index_degree, degree in enumerate(degrees):
        tx_tr = build_poly(x_tr,degree)
        tx_te = build_poly(x_te,degree)
        #weights = least_squares(y_tr,tx_tr)
            
        # mean loss over all folds
        loss_tr = []
        loss_te = []
            
        for ind, lambda_ in enumerate(lambdas):
            # losses of each fold
            losses_tr = [] 
            losses_te = []
                
            for k in range(k_fold):
                l_tr, l_te = cross_validation(y, x, k_indices, k, lambda_, degree)
                losses_tr.append(l_tr)
                losses_te.append(l_te)
            loss_tr.append(np.mean(losses_tr))
            loss_te.append(np.mean(losses_te))
        # Find optimal lambda
        min_loss_te = min(loss_te)
        indices = [i for i, v in enumerate(loss_te) if v == min_loss_te] # find indices with minimum loss
        lambda_te = lambdas[indices[-1]] # get largest lambda for which loss is minimum
            
        weights = ridge_regression(y_tr,tx_tr,lambda_te) # use optimal lambda to ridge and find weigths
            
        rmse_tr[index_degree] = np.sqrt(2*compute_loss(y_tr,tx_tr,weights))
        rmse_te[index_degree] = np.sqrt(2*compute_loss(y_te,tx_te,weights))
            # ***************************************************
    
    # bias_variance_decomposition_visualization(degrees, rmse_tr, rmse_te)

    
    # Find optimal degree for polynomial regression
    rmse_te_mean = np.expand_dims(np.mean(rmse_te, axis=0), axis=0) # rmse mean of each row, for each degree
    ind_min_degree = np.unravel_index(np.argmin(rmse_te_mean, axis=None), rmse_te_mean.shape)
    degree_te = degrees[ind_min_degree[-1]]
    
    return degree_te, lambda_te
    
# bias_variance_demo_with_cross_validation_and_ridge() 

## Generate predictions and save ouput in csv format for submission:

In [4]:
DATA_TEST_PATH = "../data/test.csv"
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [11]:
def demo():
    
    # Pre-processing of data -> delete nan and replace by mean column value
    processed_tX_train = replace_data_point_by_mean(tX)
    processed_tX_test = replace_data_point_by_mean(tX_test)
    print(np.shape(processed_tX_test))
    print('process finished')
    
    final_weights = np.array([]) # weights of all parameters with polynomial
    final_tX_test = np.array([]) # final dataset with ridge
    
    # find optimal lambda and degree for each parameters
    for i in range(processed_tX_train.shape[1]): 
        
        degree_te, lambda_te = bias_variance_demo_with_cross_validation_and_ridge(y,processed_tX_train[:,i])
        all_tx_tr = build_poly(processed_tX_train[:,i],degree_te)
        
        #1e itération
        if final_weights.size == 0:
            final_weights = ridge_regression(y,all_tx_tr,lambda_te)
            final_tX_test = build_poly(processed_tX_test[:,i],degree_te)
        
        # toutes les itérations qui suivent
        else:
            final_weights = np.concatenate((final_weights,np.array(ridge_regression(y,all_tx_tr,lambda_te))))
            final_tX_test = np.concatenate((final_tX_test,np.array(build_poly(processed_tX_test[:,i],degree_te))), axis=1)
        
        print("Bias Variance & Cross Validation & Ridge ({bi}):".format(
              bi=i))
        """
        print('Degree: ')
        print(degree_te)
        print('lambda: ')
        print(lambda_te)
        
        print(np.shape(final_weights))
        print(final_weights)
        print(np.shape(final_tX_test))
        print(final_tX_test)
        """
        
    return final_weights, final_tX_test

final_weights, final_tX_test = demo()

(568238, 30)
process finished
Bias Variance & Cross Validation & Ridge (0):
Bias Variance & Cross Validation & Ridge (1):
Bias Variance & Cross Validation & Ridge (2):
Bias Variance & Cross Validation & Ridge (3):
Bias Variance & Cross Validation & Ridge (4):
Bias Variance & Cross Validation & Ridge (5):
Bias Variance & Cross Validation & Ridge (6):
Bias Variance & Cross Validation & Ridge (7):
Bias Variance & Cross Validation & Ridge (8):
Bias Variance & Cross Validation & Ridge (9):
Bias Variance & Cross Validation & Ridge (10):
Bias Variance & Cross Validation & Ridge (11):
Bias Variance & Cross Validation & Ridge (12):
Bias Variance & Cross Validation & Ridge (13):
Bias Variance & Cross Validation & Ridge (14):
Bias Variance & Cross Validation & Ridge (15):
Bias Variance & Cross Validation & Ridge (16):
Bias Variance & Cross Validation & Ridge (17):
Bias Variance & Cross Validation & Ridge (18):
Bias Variance & Cross Validation & Ridge (19):
Bias Variance & Cross Validation & Ridge

In [12]:
print(np.shape(final_weights))
print(np.shape(final_tX_test))

(60,)
(568238, 60)


In [14]:
OUTPUT_PATH = "../data/submission002.csv"
#y_pred = predict_labels(weights, tX_test)
y_pred = predict_labels(final_weights, final_tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)